<a href="https://colab.research.google.com/github/shivam-kotwalia/data-science-project/blob/master/Visualize_CNN_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import tensorflow
import matplotlib.pyplot as plt
import numpy as np 

In [0]:
# DATA - https://www.kaggle.com/c/digit-recognizer/data
# Kaggle 
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/